<a href="https://colab.research.google.com/github/Pzof1/StudyProjects/blob/main/video_nba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install nba_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.3/285.3 kB 6.1 MB/s eta 0:00:00


In [4]:

from nba_api.stats.static import teams

nba_teams = teams.get_teams()


pacers = [team for team in nba_teams if team['abbreviation'] == 'IND'][0]
print(pacers)
pacers_id = pacers['id']
print(f'pacers_id: {pacers_id}')

{'id': 1610612754, 'full_name': 'Indiana Pacers', 'abbreviation': 'IND', 'nickname': 'Pacers', 'city': 'Indiana', 'state': 'Indiana', 'year_founded': 1976}
pacers_id: 1610612754


In [11]:
from nba_api.stats.endpoints import leaguegamefinder

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=pacers_id)

games_dict = gamefinder.get_normalized_dict()
for x in games_dict:
  print(x)
games = games_dict['LeagueGameFinderResults']#ок бесполезное говно в словаре просто один ключ и все кек
print(games)
game = games[0]
game_id = game['GAME_ID']
game_matchup = game['MATCHUP']

print(f'Searching through {len(games)} game(s) for the game_id of {game_id} where {game_matchup}')

LeagueGameFinderResults
[{'SEASON_ID': '42024', 'TEAM_ID': 1610612754, 'TEAM_ABBREVIATION': 'IND', 'TEAM_NAME': 'Indiana Pacers', 'GAME_ID': '0042400407', 'GAME_DATE': '2025-06-22', 'MATCHUP': 'IND @ OKC', 'WL': 'L', 'MIN': 240, 'PTS': 91, 'FGM': 29, 'FGA': 70, 'FG_PCT': 0.414, 'FG3M': 11, 'FG3A': 28, 'FG3_PCT': 0.393, 'FTM': 22, 'FTA': 29, 'FT_PCT': 0.759, 'OREB': 12, 'DREB': 33, 'REB': 45, 'AST': 17, 'STL': 6, 'BLK': 4, 'TOV': 21, 'PF': 24, 'PLUS_MINUS': -12.0}, {'SEASON_ID': '42024', 'TEAM_ID': 1610612754, 'TEAM_ABBREVIATION': 'IND', 'TEAM_NAME': 'Indiana Pacers', 'GAME_ID': '0042400406', 'GAME_DATE': '2025-06-19', 'MATCHUP': 'IND vs. OKC', 'WL': 'W', 'MIN': 240, 'PTS': 108, 'FGM': 38, 'FGA': 92, 'FG_PCT': 0.413, 'FG3M': 15, 'FG3A': 42, 'FG3_PCT': 0.357, 'FTM': 17, 'FTA': 25, 'FT_PCT': 0.68, 'OREB': 11, 'DREB': 35, 'REB': 46, 'AST': 23, 'STL': 16, 'BLK': 5, 'TOV': 10, 'PF': 17, 'PLUS_MINUS': 17.0}, {'SEASON_ID': '42024', 'TEAM_ID': 1610612754, 'TEAM_ABBREVIATION': 'IND', 'TEAM_NAME'

In [13]:
from nba_api.stats.endpoints import playbyplay
df = playbyplay.PlayByPlay(game_id).get_data_frames()[0]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   GAME_ID             510 non-null    object
 1   EVENTNUM            510 non-null    int64 
 2   EVENTMSGTYPE        510 non-null    int64 
 3   EVENTMSGACTIONTYPE  510 non-null    int64 
 4   PERIOD              510 non-null    int64 
 5   WCTIMESTRING        510 non-null    object
 6   PCTIMESTRING        510 non-null    object
 7   HOMEDESCRIPTION     272 non-null    object
 8   NEUTRALDESCRIPTION  11 non-null     object
 9   VISITORDESCRIPTION  259 non-null    object
 10  SCORE               118 non-null    object
 11  SCOREMARGIN         118 non-null    object
dtypes: int64(4), object(8)
memory usage: 47.9+ KB


In [ ]:
import re #TODO продумать identify_cut_events там пока что вообще пиздец
#а еще думай над def generate_ffmpeg_filter(self): мб ошибка в логике где то
from nba_api.stats.endpoints import playbyplay

class NBACutDetector:
    def __init__(self, game_id):
        self.game_id = game_id
        self.pbp_data = self._load_pbp_data()
        self.cut_intervals = []

    def _load_pbp_data(self):
        """Загрузка данных play-by-play"""
        try:
            pbp = playbyplay.PlayByPlay(self.game_id)
            return pbp.get_normalized_dict()['PlayByPlay']
        except Exception as e:
            print(f"Ошибка загрузки данных: {e}")
            return []

    def _time_to_seconds(self, time_str):
        """Конвертация времени MM:SS в секунды"""
        if not time_str:
            return 0
        try:
            minutes, seconds = time_str.split(':')
            return int(minutes) * 60 + float(seconds)
        except:
            return 0

    def identify_cut_events(self):
        """Идентификация событий для вырезания"""
        if not self.pbp_data:
            return

        for event in self.pbp_data:
            event_type = event.get('EVENTMSGTYPE', -1)
            event_time = self._time_to_seconds(event.get('PCTIMESTRING', ''))
            event_desc = event.get('HOMEDESCRIPTION', '') or event.get('VISITORDESCRIPTION', '')

            # Критерии для вырезания
            if event_type == 9:  # Тайм-ауты
                self._add_cut_interval(event_time, duration=60)

            elif event_type == 3:  # Фолы
                self._add_cut_interval(event_time, duration=30)

            elif "REBOUND" in event_desc:  # Подборы
                self._add_cut_interval(event_time, duration=10)

            elif "TIMEOUT" in event_desc:  # Дополнительная проверка тайм-аутов
                self._add_cut_interval(event_time, duration=60)

    def _add_cut_interval(self, start_time, duration):
        """Добавление интервала для вырезания"""
        end_time = start_time + duration
        self.cut_intervals.append((start_time, end_time))

    def get_cut_intervals(self):
        """Получение оптимизированных интервалов для вырезания"""
        if not self.cut_intervals:
            return []

        # Сортировка и объединение пересекающихся интервалов
        self.cut_intervals.sort(key=lambda x: x[0])
        merged = [self.cut_intervals[0]]

        for current in self.cut_intervals[1:]:
            last = merged[-1]
            if current[0] <= last[1]:
                merged[-1] = (last[0], max(last[1], current[1]))
            else:
                merged.append(current)

        return merged

    def generate_ffmpeg_filter(self):
        """Генерация фильтра для ffmpeg"""
        if not self.cut_intervals:
            return ""

        intervals = self.get_cut_intervals()
        select_filter = "select='"
        concat_filter = ",concat=n={}:v=1:a=1[outv][outa]".format(len(intervals) + 1)

        for i, (start, end) in enumerate(intervals):
            select_filter += f"between(t,{start},{end})"
            if i < len(intervals) - 1:
                select_filter += "+"

        select_filter += "'"
        return f"{select_filter} {concat_filter}"